In [ ]:
import pandas as pd
import numpy as np
from config import gkey
import numpy as np
import requests
import json
import gmaps

# Configure gmaps
gmaps.configure(api_key=gkey)

In [ ]:
#url of national capitals for scraping
url = 'https://en.wikipedia.org/wiki/List_of_national_capitals_by_latitude'

#temperaturate anamolies from https://data.giss.nasa.gov/gistemp/
zonal_temp_anamolies = pd.read_csv('C:/Users/joelb/Documents/Github/ClimateChange/ZonAnn.Ts+dSST.csv')


In [ ]:

# scrape html table from wikipediea
tables = pd.read_html(url)

#grabs the 2nd table on the webpage
city_by_hemisphere = tables[1]

#give the table column names
city_by_hemisphere.columns = ['City','Latitude','Country','Notes']

#drops the first row that is just headers
city_by_hemisphere = city_by_hemisphere.drop(0,axis=0)


#converts the latitude column into numeric
city_by_hemisphere['Latitude'] = city_by_hemisphere['Latitude'].str.replace('−','-')
city_by_hemisphere['Latitude'] = pd.to_numeric(city_by_hemisphere['Latitude'])

#if lat is negative its south, if positive its north
city_by_hemisphere['Hemisphere'] = np.where(city_by_hemisphere['Latitude']<=0, 'south', 'north')


In [ ]:

#determies what zone of the hemisphere the city is in
city_by_hemisphere['zone'] = np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>0,city_by_hemisphere['Latitude']<=24), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'EQU-24N', \
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>24,city_by_hemisphere['Latitude']<=44), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'24N-44N',
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>44,city_by_hemisphere['Latitude']<=64), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'44N-64N',
                             np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']>64,city_by_hemisphere['Latitude']<=90), \
                                            city_by_hemisphere['Hemisphere'] == 'north'),'64N-90N', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<0,city_by_hemisphere['Latitude']>=-24), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'24S-EQU', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-24,city_by_hemisphere['Latitude']>=-44), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'44S-24S', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-44,city_by_hemisphere['Latitude']>=-64), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'64S-44S', \
                            np.where(np.logical_and( \
                                    np.logical_and(city_by_hemisphere['Latitude']<-64,city_by_hemisphere['Latitude']>=-90), \
                                            city_by_hemisphere['Hemisphere'] == 'south'),'64S-44S', \
                                     ""))))))))                                  


In [ ]:
#get column headers for the year to be used later
year_header = list(zonal_temp_anamolies['Year'])

In [ ]:
#transpose the dataframe for merging
zonal_temp_anamolies = zonal_temp_anamolies.transpose()


In [ ]:

#merge the city capitals with the zonal temperature data
climate_df =  city_by_hemisphere.merge(zonal_temp_anamolies, how='left', left_on='zone', right_index=True)

In [ ]:
#rename columns to the appropriate year
climate_df.rename(columns=dict(zip(climate_df.columns[6:],year_header)),inplace=True)

In [ ]:
#drop a notes column that is not needed
climate_df = climate_df.drop('Notes',axis=1)



In [ ]:

#sums temperature anomalies over a period of years.  will be the layers on the leaflet map
climate_df['rng_1880_1910'] = climate_df.iloc[:, 5:36].sum(axis='columns')
climate_df['rng_1911_1940'] = climate_df.iloc[:, 36:66].sum(axis='columns')
climate_df['rng_1941_1970'] = climate_df.iloc[:, 66:96].sum(axis='columns')
climate_df['rng_1971_2000'] = climate_df.iloc[:, 96:126].sum(axis='columns')
climate_df['rng_2001_2017'] = climate_df.iloc[:, 126:143].sum(axis='columns')

In [ ]:
# climate_df.to_excel('C:/Users/joelb/Documents/Github/ClimateChange/test.xlsx')

In [19]:
#create lat and lgn fields for google maps geocode api
climate_df["Lat"] = ""
climate_df["Lng"] = ""

#set parameters to my gkey
params = {"key": gkey}

In [20]:
count = 0
bad_list = []

for index, row in climate_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    country = row['Country']

    # update address key value
    params['address'] = f"{city},{country}"

    # make request, print url
    cities_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:

        climate_df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        climate_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

        count +=1
        print("num records processed" + str(count))
    except:
        print("didnt copy street: " +  "city: " + city )
        bad_list.append(street)
        

num records processed1
num records processed2
num records processed3
num records processed4
num records processed5
num records processed6
num records processed7
num records processed8
num records processed9
num records processed10
num records processed11
num records processed12
num records processed13
num records processed14
num records processed15
num records processed16
num records processed17
num records processed18
num records processed19
num records processed20
num records processed21
num records processed22
num records processed23
num records processed24
num records processed25
num records processed26
num records processed27
num records processed28
num records processed29
num records processed30
num records processed31
num records processed32
num records processed33
num records processed34
num records processed35
num records processed36
num records processed37
num records processed38
num records processed39
num records processed40
num records processed41
num records processed42
n

In [22]:
#drop original lat field so it does not conflict with google lat field
climate_df = climate_df.drop('Latitude',axis=1)

climate_df

,City,Country,Hemisphere,zone,1880,1881,1882,1883,1884,1885,...,2015,2016,2017,rng_1880_1910,rng_1911_1940,rng_1941_1970,rng_1971_2000,rng_2001_2017,Lat,Lng
1,Abu Dhabi,United Arab Emirates,north,24N-44N,-0.26,-0.15,-0.09,-0.22,-0.41,-0.42,...,1.00,1.06,1.02,-8.90,-5.35,1.64,3.36,12.11,24.4539,54.3773
2,Abuja,Nigeria,north,EQU-24N,-0.19,0.06,-0.07,-0.22,-0.21,-0.08,...,0.95,0.93,0.82,-5.72,-6.07,0.07,5.38,10.12,9.07648,7.39857
3,Accra,Ghana,north,EQU-24N,-0.19,0.06,-0.07,-0.22,-0.21,-0.08,...,0.95,0.93,0.82,-5.72,-6.07,0.07,5.38,10.12,5.60372,-0.186964
4,Adamstown,Pitcairn Islands,south,44S-24S,-0.03,-0.05,0.03,0.00,-0.18,-0.30,...,0.73,0.66,0.75,-6.34,-7.10,-2.36,8.15,9.91,-25.0663,-130.1
5,Addis Ababa,Ethiopia,north,EQU-24N,-0.19,0.06,-0.07,-0.22,-0.21,-0.08,...,0.95,0.93,0.82,-5.72,-6.07,0.07,5.38,10.12,8.9806,38.7578
6,Algiers,Algeria,north,24N-44N,-0.26,-0.15,-0.09,-0.22,-0.41,-0.42,...,1.00,1.06,1.02,-8.90,-5.35,1.64,3.36,12.11,36.7538,3.05876
7,Alofi,Niue,south,24S-EQU,-0.11,0.09,-0.04,-0.16,-0.18,-0.21,...,0.89,1.03,0.77,-5.82,-6.01,-0.74,8.37,10.36,-19.0554,-169.918
8,Amman,Jordan,north,24N-44N,-0.26,-0.15,-0.09,-0.22,-0.41,-0.42,...,1.00,1.06,1.02,-8.90,-5.35,1.64,3.36,12.11,31.9454,35.9284
9,Amsterdam,Netherlands,north,44N-64N,-0.50,-0.49,-0.28,-0.61,-0.61,-0.68,...,1.44,1.42,1.35,-11.77,-1.48,0.74,9.99,17.76,52.3702,4.89517
10,Andorra la Vella,Andorra,north,24N-44N,-0.26,-0.15,-0.09,-0.22,-0.41,-0.42,...,1.00,1.06,1.02,-8.90,-5.35,1.64,3.36,12.11,42.5063,1.52184
